Test sending much data to the API and see the time difference

In [7]:
import pandas as pd
import numpy as np
import requests
import time

In [ ]:
# Define the number of rows
data_size = 10

# Generate the data
data = {
    "period": np.random.choice(["2024-12-01", "2024-11-01", "2024-10-01"], size=data_size),
    "branch": np.random.choice(["ALL", "20", "35", "12"], size=data_size),
    "agency": np.random.choice(["19", "09", "10", "11"], size=data_size),
    "partno": [f"47585451NP-{i}" for i in range(data_size)],
    "FD": np.random.uniform(50.0, 100.0, size=data_size),
    "mean_12": np.random.uniform(20.0, 40.0, size=data_size),
    "std_12": np.random.uniform(10.0, 20.0, size=data_size),
    "ub": np.random.uniform(30.0, 70.0, size=data_size)
}

# Create DataFrame
df = pd.DataFrame(data)

# Display the first few rows
display(df.tail())

,period,branch,agency,partno,FD,mean_12,std_12,ub
9995,2024-10-01,35,10,47585451NP-9995,98.416061,36.819794,17.113931,64.303620
9996,2024-10-01,20,10,47585451NP-9996,50.143286,32.719618,12.188541,36.129919
9997,2024-10-01,20,19,47585451NP-9997,75.028451,33.065671,10.821566,64.327431
9998,2024-10-01,20,11,47585451NP-9998,74.055940,33.559730,13.718711,52.330523
9999,2024-12-01,ALL,19,47585451NP-9999,57.902325,22.720715,14.423697,37.746143


In [58]:
# Define the endpoint
url = "http://172.16.1.59:18080/v1/web/test1-post"

# Send data in bulk and measure time
start_time_bulk = time.time()
bulk_data = df.to_dict(orient="records")
response = requests.post(url, json=bulk_data, params={'append': False})
if response.status_code != 200:
    print(f"Error: {response.status_code}, {response.text}")
end_time_bulk = time.time()

print(f"Time taken for bulk request: {end_time_bulk - start_time_bulk} seconds")

# Send data per hit and measure time
start_time_per_hit = time.time()
for _, row in df.iterrows():
    response = requests.post(url, json=[row.to_dict()], params={'append': True})
    if response.status_code != 200:
        print(f"Error: {response.status_code}, {response.text}")
end_time_per_hit = time.time()

print(f"Time taken for per-hit requests: {end_time_per_hit - start_time_per_hit} seconds")


Time taken for bulk request: 14.652889728546143 seconds
Time taken for per-hit requests: 117.33938217163086 seconds
